In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import tensorflow as tf
from __future__ import print_function
from sklearn import linear_model

plt.rcParams['figure.figsize'] = (10, 10)

train = pd.read_csv('train.csv')
print(train.describe())
test = pd.read_csv('test.csv')
train = train[train['GarageArea'] < 1200]
train = train[train['GrLivArea'] < 4000]
print(train.describe())

X = train.select_dtypes(include=[np.number])
print(X.shape)
y=X.pop("SalePrice")
y1=np.log(y)
print(X.shape)

print ("Skew is:", train.SalePrice.skew())
plt.hist(train.SalePrice, color='blue')
plt.show()


print ("\n Skew is:", y1.skew())
plt.hist(y1, color='blue')
plt.show()


# print(sum(X.isnull().sum() != 0))
# X["LotFrontage"]=X["LotFrontage"].fillna(X["LotFrontage"].median())
# X=X.fillna(0)
X=X.interpolate()


# print(X)
nulls = pd.DataFrame(X.isnull().sum().sort_values(ascending=False)[:25])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
print(nulls)

X1=X.drop(['Id'],axis=1)

print(X1.shape)

# print(X.LotFrontage.describe())

corr =train.corr()
# print(corr)
# The first five features are the most positively correlated with SalePrice, while the next five are the most negatively correlated.
print (corr['SalePrice'].sort_values(ascending=False)[:5], '\n')
print (corr['SalePrice'].sort_values(ascending=False)[-5:])

plt.scatter(x=X['OverallQual'], y=y1)
plt.ylabel('Sale Price')
plt.xlabel('OverallQual')
plt.show()

plt.scatter(x=X['GrLivArea'], y=y1)
plt.ylabel('Sale Price')
plt.xlabel('GrLivArea')
plt.show()

plt.scatter(x=X['GarageCars'], y=y1)
plt.ylabel('Sale Price')
plt.xlabel('GarageCars')
plt.show()

plt.scatter(x=X['GarageArea'], y=y1)
plt.ylabel('Sale Price')
plt.xlabel('GarageArea')
plt.show()



# display the previous graph again without outliers
# plt.scatter(x=X['GarageArea'], y=y1)
# plt.xlim(-200,1600)     # This forces the same scale as before
# plt.ylabel('Sale Price')
# plt.xlabel('Garage Area')
# plt.show()


categoricals = train.select_dtypes(exclude=[np.number])
#categoricals.describe()
print(categoricals.describe())

print ("Original: \n")
print (train.Street.value_counts(), "\n")
def encode1(x): return 1 if x == 'Pave' else 0
# X1['enc_street'] = pd.get_dummies(train.Street, drop_first=False)
X1['enc_street'] = train.Street.apply(encode1)

# print (train.Alley.value_counts(), "\n")

# X1['enc_Alley'] = pd.get_dummies(train.Alley, drop_first=True)

def encode(x): return 1 if x == 'Partial' else 0
X1['enc_condition'] = train.SaleCondition.apply(encode)

# X1.columns.name = None
print(X1)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X1,y1,random_state=42,test_size=0.5)




from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

lr = LinearRegression()
# lr =GradientBoostingRegressor()
# lr = KNeighborsRegressor()
# lr=RandomForestRegressor(n_estimators=100,  max_depth=1000,  
# min_samples_split=2, min_samples_leaf=1)
# lr = XGBRegressor()
# lr = Lasso(alpha=0.0001)
# lr = Ridge(alpha=0.00001)
lr.fit(X_train, y_train)

print(X_train.shape)

# alpha = 50
# lr = linear_model.Ridge(alpha=alpha)
# ridge_model = lr.fit(X_train, y_train)
# preds_ridge = ridge_model.predict(X_test)
# rms = sqrt(mean_squared_error(y_test, preds_ridge))
# print(rms)






rms = sqrt(mean_squared_error(y_test, lr.predict(X_test)))
print(rms)






test_data=test.select_dtypes(include=[np.number])
test_data=test_data.interpolate()
test_data=test_data.drop(['Id'],axis=1)
# test_data['enc_street'] = pd.get_dummies(test.Street, drop_first=True)
test_data['enc_condition'] = test.SaleCondition.apply(encode)
test_data['enc_street'] = test.Street.apply(encode1)
# test_data['enc_Alley'] = pd.get_dummies(test.Alley, drop_first=True)
print(type(test_data))



prediction = lr.predict(test_data)
pre=np.exp(prediction)
submission = pd.DataFrame({"Id": test["Id"],"SalePrice": pre})
submission.to_csv('submission.csv', index=False)

                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726   
std       1.112799    30.202904     20.645407   181.066207   456.098091   
min       1.000

<Figure size 1000x1000 with 1 Axes>


 Skew is: 0.0655782611


<Figure size 1000x1000 with 1 Axes>

              Null Count
Feature                 
YrSold                 0
BsmtFullBath           0
LowQualFinSF           0
2ndFlrSF               0
1stFlrSF               0
TotalBsmtSF            0
BsmtUnfSF              0
BsmtFinSF2             0
BsmtFinSF1             0
MasVnrArea             0
YearRemodAdd           0
YearBuilt              0
OverallCond            0
OverallQual            0
LotArea                0
LotFrontage            0
MSSubClass             0
GrLivArea              0
BsmtHalfBath           0
MoSold                 0
FullBath               0
MiscVal                0
PoolArea               0
ScreenPorch            0
3SsnPorch              0
(1452, 36)
SalePrice      1.000000
OverallQual    0.800028
GrLivArea      0.720028
GarageCars     0.652979
GarageArea     0.647456
Name: SalePrice, dtype: float64 

BsmtHalfBath    -0.036559
OverallCond     -0.080717
MSSubClass      -0.086856
EnclosedPorch   -0.129693
KitchenAbvGr    -0.139006
Name: SalePrice, dtype: float6

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

       MSZoning Street Alley LotShape LandContour Utilities LotConfig  \
count      1452   1452    91     1452        1452      1452      1452   
unique        5      2     2        4           4         2         5   
top          RL   Pave  Grvl      Reg         Lvl    AllPub    Inside   
freq       1144   1447    50      921        1307      1451      1047   

       LandSlope Neighborhood Condition1      ...      GarageType  \
count       1452         1452       1452      ...            1371   
unique         3           25          9      ...               6   
top          Gtl        NAmes       Norm      ...          Attchd   
freq        1375          225       1255      ...             865   

       GarageFinish GarageQual GarageCond PavedDrive PoolQC  Fence  \
count          1371       1371       1371       1452      5    280   
unique            3          5          5          3      3      4   
top             Unf         TA         TA          Y     Fa  MnPrv   
freq    